In [2]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leinad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import numpy
import tflearn
import tensorflow as tf
import random
import json
import pickle
from tensorflow import keras

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
with open("intents.json") as file:
    data = json.load(file)

In [9]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [10]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
try:
	model.load("model.tflearn")
except:
	model = tflearn.DNN(net)
	model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
	model.save("model.tflearn")

Training Step: 3999  | time: 0.015s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.9999 -- iter: 24/28
Training Step: 4000  | time: 0.019s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.9999 -- iter: 28/28
--
INFO:tensorflow:c:\Users\Leinad\Documents\Tech_Villain\TechandHi\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [15]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [16]:
def chat():
	print("Start Talking with the bot(type quit to stop!")
	while True:
		inp = input("You: ")
		if inp.lower() == "quit":
			break

		results = model.predict([bag_of_words(inp,words)])[0]
		results_index = numpy.argmax(results)
		tag = labels[results_index]

		if results[results_index] > 0.5:
			for tg in data["intents"]:
				if tg['tag'] == tag:
					responses = tg['responses']	
			print(random.choice(responses))
			print("\n")
		else:
			print("I didnt get that, try again")


In [18]:
chat()

Start Talking with the bot(type quit to stop!
I'm here to help. Please share what's on your mind, and I'll do my best to assist you.


I'm really sorry to hear that you're feeling this way. Remember, you're stronger than you think. Tough times don't last forever, and you have the power to overcome them. Believe in yourself and take things one step at a time.
It's completely normal to feel overwhelmed at times. Just remember, you have an incredible strength within you. You've faced challenges before and come out on top. You can do it again.
I'm here to support you through this. You're not alone. Sometimes, the darkest moments can lead to the most beautiful transformations. Keep moving forward, and you'll find your light.


Of course, this is a safe space for you to express yourself. Feel free to share whatever is on your mind. I'm here to listen.
You can talk and express yourself without any judgment here.


I didnt get that, try again
